# Result notebook of the experiments

### Classification Results

Run the below cells to load the best trained version of the models and observe their classification rate on the test set of the dataset they were trained on. 

In [26]:
### library imports

import os
from os import listdir
from os.path import isfile, join

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint


# import models
from models.lenet.lenet import *
from models.lenet.complex_lenet import *
from models.resnet.resnet import *
from models.resnet.complex_resnet import *
from models.vgg16.vgg16 import *
from models.vgg16.complex_vgg16 import *

In [27]:
### Load the Datasets

# import dataloaders
from dataloaders.cifar10_loader import load_data as load_cifar10_data
from dataloaders.cifar100_loader import load_data as load_cifar100_data
from dataloaders.cub2011_loader import load_data as load_cub200_data

### List the models we used
model_dict = {}
model_dict['LeNet'] = LeNet
model_dict['Complex_LeNet'] = ComplexLeNet
model_dict['ResNet-110'] = ResNet
model_dict['ResNet-56'] = ResNet
model_dict['Complex_ResNet-56'] = ComplexResNet
model_dict['Complex_ResNet-110'] = ComplexResNet
model_dict['VGG-16'] = VGG16
model_dict['Complex_VGG-16'] = ComplexVGG16

# initialize our dataset dictionary
dataset_dict = {}
dataset_dict['CIFAR-10'] = load_cifar10_data
dataset_dict['CIFAR-100'] = load_cifar100_data
dataset_dict['CUB-200'] = load_cub200_data



In [9]:
# Function for loading dataset

def load_data(dataset='CIFAR-10', batch_size=256, num_workers=0):
    """
    Function for loading a dataset based on the given command line arguments.

    Inputs:
        dataset - String indicating the dataset to use. Default = 'CIFAR-10'
        batch_size - Int indicating the size of the mini batches. Default = 256
        num_workers - Int indicating the number of workers to use in the
            dataloader. Default = 0 (truly deterministic)
    """

    # load the dataset if possible
    if dataset in dataset_dict:
        return dataset_dict[dataset](batch_size, num_workers)
    # alert the user if the given dataset does not exist
    else:
        assert False, "Unknown dataset name \"%s\". Available datasets are: %s" % (dataset, str(dataset_dict.keys()))

In [10]:
##Function for initializing models
def initialize_model(model='Complex_LeNet', num_classes=10, lr=3e-4, k=2):
    """
    Function for initializing a model based on the given command line arguments.

    Inputs:
        model - String indicating the model to use. Default = 'Complex_LeNet'
        num_classes - Int indicating the number of classes. Default = 10
        lr - Float indicating the optimizer learning rate. Default = 3e-4
        k - Level of anonimity. k-1 fake features are generated
            to train the discriminator. Default = 2
    """

    # initialize the model if possible
    if model == "Complex_ResNet-110" or model == "ResNet-110":
        return model_dict[model](num_classes, k, lr, num_blocks = [37,36,36])
    elif model == "Complex_ResNet-56" or model == "ResNet-56":
        return model_dict[model](num_classes, k, lr, num_blocks = [19,18,18])
    elif model in model_dict:
        return model_dict[model](num_classes, k, lr)
    # alert the user if the given model does not exist
    else:
        assert False, "Unknown model name \"%s\". Available models are: %s" % (model, str(model_dict.keys()))
        
#Initialize dictionairy for results per model
resultDict = {}

### LeNet

In [28]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: LeNet')
print('Dataset: Cifar-10')
print('Epochs: ' + str(3)) ## AANVULLEN
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 512')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CIFAR-10', 512, 0
    )
trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('LeNet', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/LeNet_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['LeNet'] = out[0]

----- MODEL SUMMARY -----
Model: LeNet
Dataset: Cifar-10
Epochs: 3
K value: 2
Learning rate: 1e3 
Batch size: 512
-------------------------
Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 1.16 GiB already allocated; 0 bytes free; 1.16 GiB reserved in total by PyTorch)

### Complex LeNet

In [12]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: Complex LeNet')
print('Dataset: Cifar-10')
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 512')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CIFAR-10', 512, 0
    )
trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('Complex_LeNet', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/Complex_LeNet_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['Complex_LeNet'] = out[0]

----- MODEL SUMMARY -----
Model: Complex LeNet
Dataset: Cifar-10
K value: 2
Learning rate: 1e3 
Batch size: 512
-------------------------
Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.2314, device='cuda:0'),
 'test_generator_loss': tensor(0.3460, device='cuda:0'),
 'test_model_loss': tensor(3.3280, device='cuda:0'),
 'test_total-loss': tensor(3.6740, device='cuda:0')}
--------------------------------------------------------------------------------

Testing successfull


### ResNet-56

In [13]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: ResNet-56')
print('Dataset: Cifar-100')
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 512')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CIFAR-100', 512, 0
    )
trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('ResNet-56', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/ResNet-56_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['ResNet-56'] = out[0]

----- MODEL SUMMARY -----
Model: ResNet-56
Dataset: Cifar-100
K value: 2
Learning rate: 1e3 
Batch size: 512
-------------------------
Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.2952, device='cuda:0'),
 'test_loss': tensor(3.1712, device='cuda:0')}
--------------------------------------------------------------------------------

Testing successfull


### Complex ResNet-56

In [14]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: Complex ResNet-56')
print('Dataset: Cifar-100')
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 512')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CIFAR-100', 512, 0
    )
trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('Complex_ResNet-56', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/Complex_ResNet-56_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['Complex_ResNet-56'] = out[0]

----- MODEL SUMMARY -----
Model: Complex ResNet-56
Dataset: Cifar-100
K value: 2
Learning rate: 1e3 
Batch size: 512
-------------------------
Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.1500, device='cuda:0'),
 'test_generator_loss': tensor(0.4614, device='cuda:0'),
 'test_model_loss': tensor(3.6868, device='cuda:0'),
 'test_total-loss': tensor(4.1482, device='cuda:0')}
--------------------------------------------------------------------------------

Testing successfull


### ResNet-110

In [15]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: ResNet-110')
print('Dataset: Cifar-100')
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 512')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CIFAR-100', 512, 0
    )
trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('ResNet-110', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/ResNet-110_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['ResNet-110'] = out[0]

----- MODEL SUMMARY -----
Model: ResNet-110
Dataset: Cifar-100
K value: 2
Learning rate: 1e3 
Batch size: 512
-------------------------
Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.2700, device='cuda:0'),
 'test_loss': tensor(3.0797, device='cuda:0')}
--------------------------------------------------------------------------------

Testing successfull


### Complex ResNet-110

In [16]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: Complex ResNet-110')
print('Dataset: Cifar-100')
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 512')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CIFAR-100', 512, 0
    )
trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('Complex_ResNet-110', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/Complex_ResNet-110_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['Complex_ResNet-110'] = out[0]

----- MODEL SUMMARY -----
Model: Complex ResNet-110
Dataset: Cifar-100
K value: 2
Learning rate: 1e3 
Batch size: 512
-------------------------
Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.1112, device='cuda:0'),
 'test_generator_loss': tensor(0.4549, device='cuda:0'),
 'test_model_loss': tensor(3.9579, device='cuda:0'),
 'test_total-loss': tensor(4.4129, device='cuda:0')}
--------------------------------------------------------------------------------

Testing successfull


### VGG-16

In [17]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: VGG-16')
print('Dataset: CUB-200')
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 256')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CUB-200', 64, 0
    )

trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('VGG-16', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/VGG-16_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['VGG-16'] = out[0]

----- MODEL SUMMARY -----
Model: VGG-16
Dataset: CUB-200
K value: 2
Learning rate: 1e3 
Batch size: 256
-------------------------


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7614, device='cuda:0'),
 'test_loss': tensor(2.5681, device='cuda:0')}
--------------------------------------------------------------------------------

Testing successfull


### Complex VGG-16

In [19]:
# print the most important arguments given by the user
print('----- MODEL SUMMARY -----')
print('Model: Complex VGG-16')
print('Dataset: CUB-200')
print('K value: 2')
print('Learning rate: 1e3 ')
print('Batch size: 256')
print('-------------------------')

    # load the data from the dataloader
num_classes, trainloader, valloader, testloader = load_data(
       'CUB-200', 64, 0
    )
trainer = pl.Trainer(
                        gpus=1 if torch.cuda.is_available() else 0,
                        max_epochs=1000,
                        progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

    # initialize the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = initialize_model('Complex_VGG-16', num_classes, 1e-3, 2)

   
# load the saved model
print('Loading model..')
model.load_state_dict(torch.load('./saved_models/Complex_VGG-16_save'))
print('Model successfully loaded')
print('Testing model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

resultDict['Complex_VGG-16'] = out[0]

----- MODEL SUMMARY -----
Model: Complex VGG-16
Dataset: CUB-200
K value: 2
Learning rate: 1e3 
Batch size: 256
-------------------------


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading model..
Model successfully loaded
Testing model..


RuntimeError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 0; 4.00 GiB total capacity; 1.07 GiB already allocated; 47.01 MiB free; 1.16 GiB reserved in total by PyTorch)

### Summary of the classification errors

In [ ]:
print('Classification errors')
print('Model name \t Original DNN \t \t Complex DNN')
print('LeNet'+  '\t'+ '\t' + '\t' + str(round(1-resultDict['LeNet']['test_acc'],3)) + '\t' + '\t' + '\t' + str(round(1-resultDict['Complex_LeNet']['test_acc'],3)))
print('ResNet-56'+  '\t' + '\t'+ str(round(1-resultDict['ResNet-56']['test_acc'],3)) + '\t' + '\t' + '\t' + str(round(1-resultDict['Complex_ResNet-56']['test_acc'],3)))
print('ResNet-110'+  '\t' + '\t'+ str(round(1-resultDict['ResNet-110']['test_acc'],3)) + '\t' + '\t' + '\t' + str(round(1-resultDict['Complex_ResNet-110']['test_acc'],3)))
print('VGG-16'+  '\t' + '\t'+ str(round(1-resultDict['VGG-16']['test_acc'],3)) + '\t' + '\t' + '\t' + str(round(1-resultDict['Complex_VGG-16']['test_acc'],3)))

# Atacker results

Run the below cells to observe the reconstruction errors for the respective models when data is intercepted by the attacker, both for the original models and for the model with obfuscated features.

In [33]:
#Import for attacker

from models.attackers.inversion_attacker_2 import *


# Initialize a dictionary for the attacker results
attackerDict = {}



### LeNet

In [36]:
classes, trainloader, valloader, testloader = load_data(
        'CIFAR-10', 128, 0
    )

trainer = pl.Trainer(
                    gpus=1 if torch.cuda.is_available() else 0,
                    max_epochs=1000,
                    progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

gan_model = initialize_model('Complex_LeNet', 10, 1e-3, 2)
gan_model.load_state_dict(torch.load('./saved_models/Complex_LeNet_save'))
generator = gan_model.encoder.generator
conv = gan_model.encoder.generator.encoding_layer

model = UNet(generator=generator, num_channels=3,enc_chs=(6,64,128,256,512), lr=1e-3, encoding_layer=conv)

print('Loading attacker model..')
model.load_state_dict(torch.load('./saved_models/Complex_ LeNetAttacker_save'))
print('Attacker model successfully loaded')
print('Testing attacker model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

attackerDict['LeNet'] = out[0]

Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading attacker model..


RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "encoding_layer.weight", "encoding_layer.bias". 
	Unexpected key(s) in state_dict: "change_channels.weight", "change_channels.bias". 

### ResNet-56

In [35]:
classes, trainloader, valloader, testloader = load_data(
        'CIFAR-100', 128, 0
    )

trainer = pl.Trainer(
                    gpus=1 if torch.cuda.is_available() else 0,
                    max_epochs=1000,
                    progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

gan_model = initialize_model('Complex_ResNet-56', 100, 1e-3, 2)
gan_model.load_state_dict(torch.load('./saved_models/Complex_ResNet-56_save'))
generator = gan_model.encoder.generator
conv = gan_model.encoder.generator.encoding_layer

model = UNet(generator=generator, enc_chs=(16,64,128,256,512), num_channels=3, lr=1e-3, encoding_layer=conv)

print('Loading attacker model..')
model.load_state_dict(torch.load('./saved_models/Complex_ResNet-56Attacker_save'))
print('Attacker model successfully loaded')
print('Testing attacker model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

attackerDict['ResNet-56'] = out[0]

Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading attacker model..


RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "encoding_layer.0.weight", "encoding_layer.1.weight", "encoding_layer.1.bias", "encoding_layer.1.running_mean", "encoding_layer.1.running_var", "encoding_layer.3.net.0.weight", "encoding_layer.3.net.1.weight", "encoding_layer.3.net.1.bias", "encoding_layer.3.net.1.running_mean", "encoding_layer.3.net.1.running_var", "encoding_layer.3.net.3.weight", "encoding_layer.3.net.4.weight", "encoding_layer.3.net.4.bias", "encoding_layer.3.net.4.running_mean", "encoding_layer.3.net.4.running_var", "encoding_layer.3.downsample.weight", "encoding_layer.3.downsample.bias", "encoding_layer.4.net.0.weight", "encoding_layer.4.net.1.weight", "encoding_layer.4.net.1.bias", "encoding_layer.4.net.1.running_mean", "encoding_layer.4.net.1.running_var", "encoding_layer.4.net.3.weight", "encoding_layer.4.net.4.weight", "encoding_layer.4.net.4.bias", "encoding_layer.4.net.4.running_mean", "encoding_layer.4.net.4.running_var", 

### ResNet-110

In [34]:
classes, trainloader, valloader, testloader = load_data(
        'CIFAR-100', 128, 0
    )

trainer = pl.Trainer(
                    gpus=1 if torch.cuda.is_available() else 0,
                    max_epochs=1000,
                    progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

gan_model = initialize_model('Complex_ResNet-110', 100, 1e-3, 2)
gan_model.load_state_dict(torch.load('./saved_models/Complex_ResNet-110_save'))
generator = gan_model.encoder.generator
conv = gan_model.encoder.generator.encoding_layer

model = UNet(generator=generator,enc_chs=(16,64,128,256,512),  num_channels=3, lr=1e-3, encoding_layer=conv)

print('Loading attacker model..')
model.load_state_dict(torch.load('./saved_models/Complex_ResNet-110Attacker_save'))
print('Attacker model successfully loaded')
print('Testing attacker model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

attackerDict['ResNet-110'] = out[0]

Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loading attacker model..


RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "encoding_layer.0.weight", "encoding_layer.1.weight", "encoding_layer.1.bias", "encoding_layer.1.running_mean", "encoding_layer.1.running_var", "encoding_layer.3.net.0.weight", "encoding_layer.3.net.1.weight", "encoding_layer.3.net.1.bias", "encoding_layer.3.net.1.running_mean", "encoding_layer.3.net.1.running_var", "encoding_layer.3.net.3.weight", "encoding_layer.3.net.4.weight", "encoding_layer.3.net.4.bias", "encoding_layer.3.net.4.running_mean", "encoding_layer.3.net.4.running_var", "encoding_layer.3.downsample.weight", "encoding_layer.3.downsample.bias", "encoding_layer.4.net.0.weight", "encoding_layer.4.net.1.weight", "encoding_layer.4.net.1.bias", "encoding_layer.4.net.1.running_mean", "encoding_layer.4.net.1.running_var", "encoding_layer.4.net.3.weight", "encoding_layer.4.net.4.weight", "encoding_layer.4.net.4.bias", "encoding_layer.4.net.4.running_mean", "encoding_layer.4.net.4.running_var", 

### VGG-16

In [ ]:
classes, trainloader, valloader, testloader = load_data(
        'CUB-200', 128, 0
    )

trainer = pl.Trainer(
                    gpus=1 if torch.cuda.is_available() else 0,
                    max_epochs=1000,
                    progress_bar_refresh_rate=1)

# seed for reproducability
pl.seed_everything(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

gan_model = initialize_model('Complex_VGG-16', num_classes, 1e-3, 2)
gan_model.load_state_dict(torch.load('./saved_models/Complex_VGG-16_save'))
generator = gan_model.encoder.generator
conv = gan_model.encoder.generator.encoding_layer

model = UNet(generator=generator, num_channels=3, lr=1e-3, encoding_layer=conv)

print('Loading attacker model..')
model.load_state_dict(torch.load('./saved_models/(Complex)VGG-16Attacker_save'))
print('Attacker model successfully loaded')
print('Testing attacker model..')
out = trainer.test(model=model, test_dataloaders=testloader)
print('Testing successfull')

attackerDict['VGG-16'] = out[0]

### Summary of the attacker results

In [ ]:
print('Reconstruction errors')
print('Model name \t Original DNN \t \t Complex DNN')
print('LeNet'+  '\t'+ '\t' + '\t' + str(round(attackerDict['LeNet']['Non-obfuscated - reconstruction_error'],3)) + '\t' + '\t' + '\t' + str(round(attackerDict['LeNet']['Obfuscated - reconstruction_error'],3)))
print('ResNet-56'+  '\t' + '\t'+ str(round(attackerDict['ResNet-56']['Non-obfuscated - reconstruction_error'],3)) + '\t' + '\t' + '\t' + str(round(attackerDict['ResNet-56']['Obfuscated - reconstruction_error'],3)))
print('ResNet-110'+  '\t' + '\t'+ str(round(attackerDict['ResNet-110']['Non-obfuscated - reconstruction_error'],3)) + '\t' + '\t' + '\t' + str(round(attackerDict['ResNet-110']['Obfuscated - reconstruction_error'],3)))
print('VGG-16'+  '\t' + '\t'+ str(round(attackerDict['VGG-16']['Non-obfuscated - reconstruction_error'],3)) + '\t' + '\t' + '\t' + str(round(attackerDict['ResNet-110']['Obfuscated - reconstruction_error'],3)))